In [ ]:
import os
import sys
import math

## Numerical
import numpy as np

## Mesh and Graphics
import igl
import meshplot as mp

In [ ]:
mesh_dir = './example_data/processed_data/meshes'
match_dir = './example_data/processed_data/match_results'

In [ ]:
def readMesh(fn):
    v,f = igl.read_triangle_mesh(fn)
    v -= v.mean(axis=0)
    v /= np.sqrt(area(v,f))
    return v,f


def area(v,f):
    a,b,c = f.T
    areas = .5*np.cross(v[b]-v[a], v[c]-v[a])
    face_areas = np.linalg.norm(areas,axis=-1)
    return face_areas.sum()


def orthogonalProcrustes(X,Y):
    U,_,VT = np.linalg.svd(X.T @ Y)
    R  = (VT.T).dot(U.T)
    return R

In [ ]:
fn1 = "Q03"
fn2 = "Q04"

i,j = np.load(os.path.join(match_dir, "{0}_{1}.npy".format(fn1, fn2)))
v1,f1 = readMesh(os.path.join(mesh_dir, fn1 + ".ply"))
v2,f2 = readMesh(os.path.join(mesh_dir, fn2 + ".ply"))

R  = orthogonalProcrustes(v2[j], v1[i])
v1 = v1 @ R 
 
scalar  = v2.copy()
scalar -= v2.min(axis=0)
scalar /= v2.max(axis=0)
scalar *= .6 # this can be tweaked

fig = mp.subplot(v1, f1, c=scalar[j], s=[2, 2, 0])
mp.subplot(v2, f2, c=scalar[i], s=[2, 2, 1], data=fig)